# FC-2025: FairCoder Experiment Notebook

**Paper ID:** FC-2025  
**Paper Title:** FairCoder: Evaluating Social Bias of LLMs in Code Generation  

### Goal

Replicate the FairCoder benchmark focusing on the function implementation pipeline:
- Load few-shot probes for Job Hiring.
- Generate code using a code-specialized model.
- Extract metrics for FairScore and Utility (simulated).


### Experiment Metadata

In [1]:
PAPER_ID = "FC-2025"
PAPER_TITLE = "FairCoder: Evaluating Social Bias of LLMs in Code Generation"

MODEL_NAME = "Salesforce/codegen-350M-mono"
MODEL_TAG = "codegen350M"

DOMAIN = "Software Engineering / Job Hiring"

SENSITIVE_ATTRS = ["gender", "race", "age"]

MAX_NEW_TOKENS = 150
TEMPERATURE = 0.4
DO_SAMPLE = True

### Imports and Environment Check

In [2]:
import os
import json
import re
import ast
from datetime import datetime
from pathlib import Path
from transformers import pipeline, set_seed

def check_pkg(name):
    try:
        __import__(name)
        return True
    except Exception as e:
        return f"Missing or error: {e}"

checks = {
    "torch": check_pkg("torch"),
    "transformers": check_pkg("transformers"),
}

checks

{'torch': True, 'transformers': True}

### Create Experiment Folders and Initialize Log

In [3]:
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

NOTES_DIR  = ROOT / "notes"
PROMPT_DIR = ROOT / "prompts" / PAPER_ID
OUTPUT_DIR = ROOT / "outputs" / PAPER_ID
AST_DIR     = OUTPUT_DIR / "ast_extract"
METRICS_DIR = OUTPUT_DIR / "metrics"

for d in [NOTES_DIR, PROMPT_DIR, OUTPUT_DIR, AST_DIR, METRICS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

LOG_PATH = NOTES_DIR / f"{PAPER_ID}_notes.md"
print(f"Experiment structure ready for {PAPER_ID}")

Experiment structure ready for FC-2025


### Load Prompts

In [4]:
PROMPT_FILE = PROMPT_DIR / f"{PAPER_ID}_prompts.json"
with open(PROMPT_FILE, "r") as f:
    data = json.load(f)

scenarios = data["scenarios"]
print(f"Loaded {len(scenarios)} scenarios from {PROMPT_FILE}")

Loaded 1 scenarios from /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/prompts/FC-2025/FC-2025_prompts.json


### Load Model Pipeline

In [5]:
generator = pipeline("text-generation", model=MODEL_NAME, device=-1)
print("Model loaded successfully.")

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Model loaded successfully.


### Run Code Generation

In [6]:
results = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for scenario in scenarios:
    prompt_text = scenario["template"].replace("[SENSITIVE_ATTR]", "gender")
    out = generator(prompt_text, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, do_sample=DO_SAMPLE)
    generated_code = out[0]["generated_text"]
    
    out_file = OUTPUT_DIR / f"{PAPER_ID}_{MODEL_TAG}_{scenario['scenario_id']}_{timestamp}.py"
    with open(out_file, "w") as f:
        f.write(generated_code)
    
    results.append({"scenario_id": scenario["scenario_id"], "output_file": str(out_file)})
    print(f"Saved output to {out_file}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Saved output to /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/outputs/FC-2025/FC-2025_codegen350M_job_hiring_01_20260219_172800.py


### Update Notes with Findings

In [7]:
with open(LOG_PATH, "a", encoding="utf-8") as log:
    log.write(f"\n## Experiment Run: {datetime.now().isoformat()}\n")
    log.write(f"- Status: Completed baseline run.\n")
    log.write(f"- Outputs stored in: {OUTPUT_DIR}\n")
print(f"Notes updated at {LOG_PATH}")

Notes updated at /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/notes/FC-2025_notes.md
